##### Module import

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor # 다중공선성

In [2]:
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

##### Data import

In [3]:
print(os.getcwd())

c:\Users\Owner\Desktop\윤태준\고용패널\고용패널 파이썬 분석\2023_04_06_변수추가


In [4]:
df = pd.read_csv('2023_04_06_data_step.csv', encoding='cp949')
df.columns

Index(['pid', 'majorcat', 'province', 'area', 'sex', 'age', 'a003', 'a010',
       'a141', 'a144', 'a146', 'd016', 'd131', 'd134', 'd136', 'f073', 'f074',
       'h001', 'q001', 'q015', 'q016', 'q017', 'p001', 'First_Work', 'year',
       'Resign', 'a038', 'a039', 'a043', 'a045', 'g191a046', 'a048', 'a392',
       'a140', 'd130', 'Admission', 'Graduation', 'Cur', 'Cur_Work_date',
       'Start_Work1_date', 'End_Work1_date', 'Work_to_Cur', 'Start_to_End',
       'FirstWorkPeriod', 'LongWork', 'LongWork2', 'major_same', 'major_help',
       'big_company', 'work_sati', 'score', 'q001G', 'p001G', 'provinceG',
       'major_sameG', 'major_helpG', 'majorcatG', 'q015G', 'q016G', 'q017G',
       'big_companyG', 'work_satiG', '나이', '학점', '성별', '건강상태', '결혼상태', '대학지역',
       '전공일치', '본전공', '재학중일자리경험', '삶의만족도_개인', '삶의만족도_관계', '삶의만족도_소속집단', '종사자수',
       '업무만족도'],
      dtype='object')

In [5]:
df_1 = df[['pid', 'majorcat', 'province', 'area', 'sex', 'age', 'a003', 'a010',
       'a141', 'a144', 'a146', 'd016', 'd131', 'd134', 'd136', 'f073', 'f074',
       'h001', 'q001', 'q015', 'q016', 'q017', 'p001', 'First_Work', 'year',
       'Resign', 'Admission', 'Graduation', 'Cur', 'Cur_Work_date',
       'Start_Work1_date', 'End_Work1_date', 'Work_to_Cur', 'Start_to_End',
       'FirstWorkPeriod', 'LongWork', 'LongWork2', 'major_same', 'major_help',
       'big_company', 'work_sati', 'score', 'q001G', 'p001G', 'provinceG',
       'major_sameG', 'major_helpG', 'majorcatG', 'q015G', 'q016G', 'q017G',
       'big_companyG', 'work_satiG',
       '나이', '성별', '건강상태', '결혼상태', '대학지역',  '전공일치', '본전공', '학점',
       '재학중일자리경험', '삶의만족도_개인', '삶의만족도_관계', '삶의만족도_소속집단',
       '종사자수', '업무만족도']].copy()

모델 Logistic Regression
1. 인구통계학적 특성
2. 인구통계학 + 전공 연관성, 대학 지역, 학점, 재학 중 경험 일자리(완전 대학 관련)
3. 인구통계학 + 전공 연관성 + 만족도(개인, 관계, 소속 집단, 업무만족도)
---------------------------------------------------
- 근속 종속변수 : :LongWork, LongWork2가 존재
    - LongWork2가 2년 이상 일한 사람 
    - 2년 ~ 3년 사이 기간을 분석하기로 했으니 Target : LongWork2로 지정해서 사용

##### 다변량 모델

###### 3번 삶의 만족도 로지스틱 

###### 소속집단 빼고

In [14]:
model_3 = sm.Logit.from_formula('''LongWork2 ~ 나이 + C(성별,Treatment('여자')) + C(건강상태,Treatment('건강하지않다')) 
    + C(결혼상태, Treatment('미혼'))+ C(대학지역,Treatment('비수도권')) + C(전공일치,Treatment('맞지않다')) 
    + 학점 + C(재학중일자리경험,Treatment('없다')) + C(삶의만족도_개인,Treatment('그렇지않다')) 
    + C(삶의만족도_관계, Treatment('그렇지않다')) + C(업무만족도,Treatment('그렇지않다')) ''', df_1).fit()
model_3.summary()

Optimization terminated successfully.
         Current function value: 0.519473
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              LongWork2   No. Observations:                 5568
Model:                          Logit   Df Residuals:                     5551
Method:                           MLE   Df Model:                           16
Date:                Tue, 11 Apr 2023   Pseudo R-squ.:                  0.1106
Time:                        19:46:12   Log-Likelihood:                -2892.4
converged:                       True   LL-Null:                       -3252.2
Covariance Type:            nonrobust   LLR p-value:                8.979e-143
===========================================================================================================
                                              coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                                 -10.2698      0.621    -16.540      0.000     -11.487      -9.053
C(성별, Treatment('여자'))[T.남자]                0.2275      0.077      2.960      0.003       0.077       0.378
C(건강상태, Treatment('건강하지않다'))[T.건강하다]        0.0896      0.117      0.765      0.444      -0.140       0.319
C(건강상태, Treatment('건강하지않다'))[T.보통이다]        0.0233      0.123      0.190      0.849      -0.217       0.263
C(결혼상태, Treatment('미혼'))[T.기혼]              0.2692      0.213      1.266      0.206      -0.148       0.686
C(대학지역, Treatment('비수도권'))[T.수도권]           0.2322      0.066      3.531      0.000       0.103       0.361
C(전공일치, Treatment('맞지않다'))[T.보통이다]          0.5346      0.085      6.277      0.000       0.368       0.702
C(전공일치, Treatment('맞지않다'))[T.잘맞다]           0.8894      0.078     11.359      0.000       0.736       1.043
C(재학중일자리경험, Treatment('없다'))[T.있다]         -0.1375      0.067     -2.057      0.040      -0.268      -0.006
C(삶의만족도_개인, Treatment('그렇지않다'))[T.그렇다]      0.2250      0.134      1.674      0.094      -0.038       0.488
C(삶의만족도_개인, Treatment('그렇지않다'))[T.보통이다]     0.0572      0.142      0.403      0.687      -0.221       0.336
C(삶의만족도_관계, Treatment('그렇지않다'))[T.그렇다]      0.1533      0.153      1.005      0.315      -0.146       0.452
C(삶의만족도_관계, Treatment('그렇지않다'))[T.보통이다]     0.0820      0.161      0.509      0.611      -0.234       0.398
C(업무만족도, Treatment('그렇지않다'))[T.그렇다]         0.7913      0.103      7.661      0.000       0.589       0.994
C(업무만족도, Treatment('그렇지않다'))[T.보통이다]        0.3399      0.107      3.185      0.001       0.131       0.549
나이                                          0.3053      0.022     13.684      0.000       0.262       0.349
학점                                         -0.0623      0.071     -0.880      0.379      -0.201       0.077
===========================================================================================================
"""

In [16]:
print('==================================Odds Ratio=========================================')
# 오즈비 계산(Confidence interval 포함)
odds_ratios_3 = pd.DataFrame(
    {
        "OR": model_3.params,
        "Lower CI": model_3.conf_int()[0],
        "Upper CI": model_3.conf_int()[1],
    }
)
odds_ratios_3 = np.exp(odds_ratios_3)
odds_ratios_3

==================================Odds Ratio=========================================


,OR,Lower CI,Upper CI
Intercept,0.000035,0.000010,0.000117
"C(성별, Treatment('여자'))[T.남자]",1.255453,1.079882,1.459569
"C(건강상태, Treatment('건강하지않다'))[T.건강하다]",1.093740,0.869488,1.375831
"C(건강상태, Treatment('건강하지않다'))[T.보통이다]",1.023555,0.804987,1.301468
"C(결혼상태, Treatment('미혼'))[T.기혼]",1.308923,0.862802,1.985715
"C(대학지역, Treatment('비수도권'))[T.수도권]",1.261341,1.108839,1.434817
"C(전공일치, Treatment('맞지않다'))[T.보통이다]",1.706770,1.444377,2.016831
"C(전공일치, Treatment('맞지않다'))[T.잘맞다]",2.433569,2.087357,2.837204
"C(재학중일자리경험, Treatment('없다'))[T.있다]",0.871555,0.764536,0.993555
"C(삶의만족도_개인, Treatment('그렇지않다'))[T.그렇다]",1.252323,0.962287,1.629776


###### 관계 빼고

In [6]:
model_3_no_pop = sm.Logit.from_formula('''LongWork2 ~ 나이 + C(성별,Treatment('여자')) + C(건강상태,Treatment('건강하지않다')) 
    + C(결혼상태, Treatment('미혼'))+ C(대학지역,Treatment('비수도권')) + C(전공일치,Treatment('맞지않다')) 
    + 학점 + C(재학중일자리경험,Treatment('없다')) + C(삶의만족도_개인,Treatment('그렇지않다')) 
    + C(삶의만족도_소속집단, Treatment('그렇지않다')) + C(업무만족도,Treatment('그렇지않다')) ''', df_1).fit()
model_3_no_pop.summary()

Optimization terminated successfully.
         Current function value: 0.518852
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              LongWork2   No. Observations:                 5568
Model:                          Logit   Df Residuals:                     5551
Method:                           MLE   Df Model:                           16
Date:                Tue, 11 Apr 2023   Pseudo R-squ.:                  0.1117
Time:                        22:23:52   Log-Likelihood:                -2889.0
converged:                       True   LL-Null:                       -3252.2
Covariance Type:            nonrobust   LLR p-value:                3.039e-144
=============================================================================================================
                                                coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                   -10.0751      0.619    -16.266      0.000     -11.289      -8.861
C(성별, Treatment('여자'))[T.남자]                  0.2296      0.077      2.988      0.003       0.079       0.380
C(건강상태, Treatment('건강하지않다'))[T.건강하다]          0.1136      0.117      0.970      0.332      -0.116       0.343
C(건강상태, Treatment('건강하지않다'))[T.보통이다]          0.0283      0.123      0.231      0.817      -0.212       0.269
C(결혼상태, Treatment('미혼'))[T.기혼]                0.2555      0.213      1.199      0.231      -0.162       0.673
C(대학지역, Treatment('비수도권'))[T.수도권]             0.2351      0.066      3.574      0.000       0.106       0.364
C(전공일치, Treatment('맞지않다'))[T.보통이다]            0.5341      0.085      6.270      0.000       0.367       0.701
C(전공일치, Treatment('맞지않다'))[T.잘맞다]             0.8957      0.078     11.424      0.000       0.742       1.049
C(재학중일자리경험, Treatment('없다'))[T.있다]           -0.1315      0.067     -1.969      0.049      -0.262      -0.001
C(삶의만족도_개인, Treatment('그렇지않다'))[T.그렇다]        0.4913      0.133      3.688      0.000       0.230       0.752
C(삶의만족도_개인, Treatment('그렇지않다'))[T.보통이다]       0.2295      0.139      1.645      0.100      -0.044       0.503
C(삶의만족도_소속집단, Treatment('그렇지않다'))[T.그렇다]     -0.3833      0.135     -2.835      0.005      -0.648      -0.118
C(삶의만족도_소속집단, Treatment('그렇지않다'))[T.보통이다]    -0.3272      0.137     -2.390      0.017      -0.596      -0.059
C(업무만족도, Treatment('그렇지않다'))[T.그렇다]           0.8292      0.104      7.955      0.000       0.625       1.033
C(업무만족도, Treatment('그렇지않다'))[T.보통이다]          0.3615      0.107      3.376      0.001       0.152       0.571
나이                                            0.3046      0.022     13.637      0.000       0.261       0.348
학점                                           -0.0580      0.071     -0.819      0.413      -0.197       0.081
=============================================================================================================
"""

In [7]:
print('==================================Odds Ratio=========================================')
# 오즈비 계산(Confidence interval 포함)
odds_ratios_3_no_pop = pd.DataFrame(
    {
        "OR": model_3_no_pop.params,
        "Lower CI": model_3_no_pop.conf_int()[0],
        "Upper CI": model_3_no_pop.conf_int()[1],
    }
)
odds_ratios_3_no_pop = np.exp(odds_ratios_3_no_pop)
odds_ratios_3_no_pop

==================================Odds Ratio=========================================


,OR,Lower CI,Upper CI
Intercept,0.000042,0.000013,0.000142
"C(성별, Treatment('여자'))[T.남자]",1.258138,1.082187,1.462696
"C(건강상태, Treatment('건강하지않다'))[T.건강하다]",1.120354,0.890515,1.409514
"C(건강상태, Treatment('건강하지않다'))[T.보통이다]",1.028710,0.808966,1.308143
"C(결혼상태, Treatment('미혼'))[T.기혼]",1.291135,0.850232,1.960678
"C(대학지역, Treatment('비수도권'))[T.수도권]",1.265058,1.112006,1.439174
"C(전공일치, Treatment('맞지않다'))[T.보통이다]",1.705842,1.443568,2.015766
"C(전공일치, Treatment('맞지않다'))[T.잘맞다]",2.449086,2.100227,2.855894
"C(재학중일자리경험, Treatment('없다'))[T.있다]",0.876779,0.769198,0.999406
"C(삶의만족도_개인, Treatment('그렇지않다'))[T.그렇다]",1.634370,1.258831,2.121940


###### 개인 빼고

In [8]:
model_3_no_per = sm.Logit.from_formula('''LongWork2 ~ 나이 + C(성별,Treatment('여자')) + C(건강상태,Treatment('건강하지않다')) 
    + C(결혼상태, Treatment('미혼'))+ C(대학지역,Treatment('비수도권')) + C(전공일치,Treatment('맞지않다')) 
    + 학점 + C(재학중일자리경험,Treatment('없다')) + C(삶의만족도_관계,Treatment('그렇지않다')) 
    + C(삶의만족도_소속집단, Treatment('그렇지않다')) + C(업무만족도,Treatment('그렇지않다')) ''', df_1).fit()
model_3_no_per.summary()

Optimization terminated successfully.
         Current function value: 0.518848
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              LongWork2   No. Observations:                 5568
Model:                          Logit   Df Residuals:                     5551
Method:                           MLE   Df Model:                           16
Date:                Tue, 11 Apr 2023   Pseudo R-squ.:                  0.1117
Time:                        22:27:01   Log-Likelihood:                -2888.9
converged:                       True   LL-Null:                       -3252.2
Covariance Type:            nonrobust   LLR p-value:                2.967e-144
=============================================================================================================
                                                coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                   -10.1640      0.620    -16.384      0.000     -11.380      -8.948
C(성별, Treatment('여자'))[T.남자]                  0.2503      0.077      3.255      0.001       0.100       0.401
C(건강상태, Treatment('건강하지않다'))[T.건강하다]          0.1837      0.114      1.616      0.106      -0.039       0.407
C(건강상태, Treatment('건강하지않다'))[T.보통이다]          0.0772      0.121      0.637      0.524      -0.160       0.315
C(결혼상태, Treatment('미혼'))[T.기혼]                0.2904      0.213      1.366      0.172      -0.126       0.707
C(대학지역, Treatment('비수도권'))[T.수도권]             0.2355      0.066      3.578      0.000       0.106       0.364
C(전공일치, Treatment('맞지않다'))[T.보통이다]            0.5384      0.085      6.319      0.000       0.371       0.705
C(전공일치, Treatment('맞지않다'))[T.잘맞다]             0.9041      0.078     11.539      0.000       0.750       1.058
C(재학중일자리경험, Treatment('없다'))[T.있다]           -0.1463      0.067     -2.186      0.029      -0.277      -0.015
C(삶의만족도_관계, Treatment('그렇지않다'))[T.그렇다]        0.6190      0.164      3.776      0.000       0.298       0.940
C(삶의만족도_관계, Treatment('그렇지않다'))[T.보통이다]       0.3741      0.167      2.239      0.025       0.047       0.702
C(삶의만족도_소속집단, Treatment('그렇지않다'))[T.그렇다]     -0.4830      0.148     -3.272      0.001      -0.772      -0.194
C(삶의만족도_소속집단, Treatment('그렇지않다'))[T.보통이다]    -0.4220      0.146     -2.899      0.004      -0.707      -0.137
C(업무만족도, Treatment('그렇지않다'))[T.그렇다]           0.8459      0.104      8.125      0.000       0.642       1.050
C(업무만족도, Treatment('그렇지않다'))[T.보통이다]          0.3641      0.107      3.402      0.001       0.154       0.574
나이                                            0.3039      0.022     13.622      0.000       0.260       0.348
학점                                           -0.0648      0.071     -0.915      0.360      -0.204       0.074
=============================================================================================================
"""

In [9]:
print('==================================Odds Ratio=========================================')
# 오즈비 계산(Confidence interval 포함)
odds_ratios_3_no_per = pd.DataFrame(
    {
        "OR": model_3_no_per.params,
        "Lower CI": model_3_no_per.conf_int()[0],
        "Upper CI": model_3_no_per.conf_int()[1],
    }
)
odds_ratios_3_no_per = np.exp(odds_ratios_3_no_per)
odds_ratios_3_no_per

==================================Odds Ratio=========================================


,OR,Lower CI,Upper CI
Intercept,0.000039,0.000011,0.000130
"C(성별, Treatment('여자'))[T.남자]",1.284472,1.104744,1.493439
"C(건강상태, Treatment('건강하지않다'))[T.건강하다]",1.201697,0.961588,1.501760
"C(건강상태, Treatment('건강하지않다'))[T.보통이다]",1.080217,0.851853,1.369800
"C(결혼상태, Treatment('미혼'))[T.기혼]",1.336915,0.881359,2.027937
"C(대학지역, Treatment('비수도권'))[T.수도권]",1.265483,1.112364,1.439680
"C(전공일치, Treatment('맞지않다'))[T.보통이다]",1.713217,1.449751,2.024562
"C(전공일치, Treatment('맞지않다'))[T.잘맞다]",2.469593,2.118049,2.879486
"C(재학중일자리경험, Treatment('없다'))[T.있다]",0.863902,0.757702,0.984987
"C(삶의만족도_관계, Treatment('그렇지않다'))[T.그렇다]",1.857027,1.346725,2.560692


###### 관계, 소속집단 빼고

In [10]:
model_3_per_pop = sm.Logit.from_formula('''LongWork2 ~ 나이 + C(성별,Treatment('여자')) + C(건강상태,Treatment('건강하지않다')) 
    + C(결혼상태, Treatment('미혼'))+ C(대학지역,Treatment('비수도권')) + C(전공일치,Treatment('맞지않다')) 
    + 학점 + C(재학중일자리경험,Treatment('없다')) + C(삶의만족도_개인,Treatment('그렇지않다')) 
    + C(업무만족도,Treatment('그렇지않다')) ''', df_1).fit()
model_3_per_pop.summary()

Optimization terminated successfully.
         Current function value: 0.519577
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              LongWork2   No. Observations:                 5568
Model:                          Logit   Df Residuals:                     5553
Method:                           MLE   Df Model:                           14
Date:                Tue, 11 Apr 2023   Pseudo R-squ.:                  0.1104
Time:                        22:31:23   Log-Likelihood:                -2893.0
converged:                       True   LL-Null:                       -3252.2
Covariance Type:            nonrobust   LLR p-value:                3.092e-144
===========================================================================================================
                                              coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                                 -10.2169      0.617    -16.555      0.000     -11.427      -9.007
C(성별, Treatment('여자'))[T.남자]                0.2245      0.077      2.925      0.003       0.074       0.375
C(건강상태, Treatment('건강하지않다'))[T.건강하다]        0.0953      0.117      0.816      0.415      -0.134       0.324
C(건강상태, Treatment('건강하지않다'))[T.보통이다]        0.0225      0.122      0.184      0.854      -0.218       0.263
C(결혼상태, Treatment('미혼'))[T.기혼]              0.2658      0.213      1.250      0.211      -0.151       0.683
C(대학지역, Treatment('비수도권'))[T.수도권]           0.2317      0.066      3.525      0.000       0.103       0.361
C(전공일치, Treatment('맞지않다'))[T.보통이다]          0.5345      0.085      6.279      0.000       0.368       0.701
C(전공일치, Treatment('맞지않다'))[T.잘맞다]           0.8905      0.078     11.375      0.000       0.737       1.044
C(재학중일자리경험, Treatment('없다'))[T.있다]         -0.1338      0.067     -2.004      0.045      -0.265      -0.003
C(삶의만족도_개인, Treatment('그렇지않다'))[T.그렇다]      0.3038      0.112      2.701      0.007       0.083       0.524
C(삶의만족도_개인, Treatment('그렇지않다'))[T.보통이다]     0.0909      0.127      0.715      0.475      -0.158       0.340
C(업무만족도, Treatment('그렇지않다'))[T.그렇다]         0.7960      0.103      7.714      0.000       0.594       0.998
C(업무만족도, Treatment('그렇지않다'))[T.보통이다]        0.3424      0.107      3.210      0.001       0.133       0.551
나이                                          0.3054      0.022     13.689      0.000       0.262       0.349
학점                                         -0.0611      0.071     -0.862      0.389      -0.200       0.078
===========================================================================================================
"""

In [11]:
print('==================================Odds Ratio=========================================')
# 오즈비 계산(Confidence interval 포함)
odds_ratios_3_per_pop = pd.DataFrame(
    {
        "OR": model_3_per_pop.params,
        "Lower CI": model_3_per_pop.conf_int()[0],
        "Upper CI": model_3_per_pop.conf_int()[1],
    }
)
odds_ratios_3_per_pop = np.exp(odds_ratios_3_per_pop)
odds_ratios_3_per_pop

==================================Odds Ratio=========================================


,OR,Lower CI,Upper CI
Intercept,0.000037,0.000011,0.000123
"C(성별, Treatment('여자'))[T.남자]",1.251666,1.076845,1.454869
"C(건강상태, Treatment('건강하지않다'))[T.건강하다]",1.100034,0.874783,1.383286
"C(건강상태, Treatment('건강하지않다'))[T.보통이다]",1.022781,0.804513,1.300265
"C(결혼상태, Treatment('미혼'))[T.기혼]",1.304429,0.859767,1.979066
"C(대학지역, Treatment('비수도권'))[T.수도권]",1.260726,1.108328,1.434079
"C(전공일치, Treatment('맞지않다'))[T.보통이다]",1.706607,1.444374,2.016450
"C(전공일치, Treatment('맞지않다'))[T.잘맞다]",2.436377,2.089796,2.840437
"C(재학중일자리경험, Treatment('없다'))[T.있다]",0.874797,0.767541,0.997040
"C(삶의만족도_개인, Treatment('그렇지않다'))[T.그렇다]",1.355010,1.086883,1.689282


###### 개인, 관계빼고

In [13]:
model_3_per_rel = sm.Logit.from_formula('''LongWork2 ~ 나이 + C(성별,Treatment('여자')) + C(건강상태,Treatment('건강하지않다')) 
    + C(결혼상태, Treatment('미혼'))+ C(대학지역,Treatment('비수도권')) + C(전공일치,Treatment('맞지않다')) 
    + 학점 + C(재학중일자리경험,Treatment('없다')) + C(삶의만족도_소속집단,Treatment('그렇지않다')) 
    + C(업무만족도,Treatment('그렇지않다')) ''', df_1).fit()
model_3_per_rel.summary()

Optimization terminated successfully.
         Current function value: 0.520253
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              LongWork2   No. Observations:                 5568
Model:                          Logit   Df Residuals:                     5553
Method:                           MLE   Df Model:                           14
Date:                Tue, 11 Apr 2023   Pseudo R-squ.:                  0.1093
Time:                        22:38:42   Log-Likelihood:                -2896.8
converged:                       True   LL-Null:                       -3252.2
Covariance Type:            nonrobust   LLR p-value:                1.253e-142
=============================================================================================================
                                                coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                   -10.0149      0.616    -16.265      0.000     -11.222      -8.808
C(성별, Treatment('여자'))[T.남자]                  0.2406      0.077      3.137      0.002       0.090       0.391
C(건강상태, Treatment('건강하지않다'))[T.건강하다]          0.2332      0.113      2.067      0.039       0.012       0.454
C(건강상태, Treatment('건강하지않다'))[T.보통이다]          0.0962      0.121      0.797      0.426      -0.140       0.333
C(결혼상태, Treatment('미혼'))[T.기혼]                0.2933      0.212      1.381      0.167      -0.123       0.710
C(대학지역, Treatment('비수도권'))[T.수도권]             0.2293      0.066      3.492      0.000       0.101       0.358
C(전공일치, Treatment('맞지않다'))[T.보통이다]            0.5435      0.085      6.393      0.000       0.377       0.710
C(전공일치, Treatment('맞지않다'))[T.잘맞다]             0.9094      0.078     11.621      0.000       0.756       1.063
C(재학중일자리경험, Treatment('없다'))[T.있다]           -0.1325      0.067     -1.986      0.047      -0.263      -0.002
C(삶의만족도_소속집단, Treatment('그렇지않다'))[T.그렇다]     -0.1090      0.114     -0.955      0.340      -0.333       0.115
C(삶의만족도_소속집단, Treatment('그렇지않다'))[T.보통이다]    -0.2033      0.124     -1.640      0.101      -0.446       0.040
C(업무만족도, Treatment('그렇지않다'))[T.그렇다]           0.8445      0.104      8.127      0.000       0.641       1.048
C(업무만족도, Treatment('그렇지않다'))[T.보통이다]          0.3637      0.107      3.406      0.001       0.154       0.573
나이                                            0.3053      0.022     13.709      0.000       0.262       0.349
학점                                           -0.0642      0.071     -0.907      0.364      -0.203       0.074
=============================================================================================================
"""

In [14]:
print('==================================Odds Ratio=========================================')
# 오즈비 계산(Confidence interval 포함)
odds_ratios_3_per_rel = pd.DataFrame(
    {
        "OR": model_3_per_rel.params,
        "Lower CI": model_3_per_rel.conf_int()[0],
        "Upper CI": model_3_per_rel.conf_int()[1],
    }
)
odds_ratios_3_per_rel = np.exp(odds_ratios_3_per_rel)
odds_ratios_3_per_rel

==================================Odds Ratio=========================================


,OR,Lower CI,Upper CI
Intercept,0.000045,0.000013,0.000150
"C(성별, Treatment('여자'))[T.남자]",1.271987,1.094449,1.478325
"C(건강상태, Treatment('건강하지않다'))[T.건강하다]",1.262618,1.012183,1.575017
"C(건강상태, Treatment('건강하지않다'))[T.보통이다]",1.100925,0.869051,1.394666
"C(결혼상태, Treatment('미혼'))[T.기혼]",1.340840,0.884242,2.033212
"C(대학지역, Treatment('비수도권'))[T.수도권]",1.257744,1.105835,1.430521
"C(전공일치, Treatment('맞지않다'))[T.보통이다]",1.722073,1.457732,2.034350
"C(전공일치, Treatment('맞지않다'))[T.잘맞다]",2.482757,2.129724,2.894310
"C(재학중일자리경험, Treatment('없다'))[T.있다]",0.875934,0.768606,0.998249
"C(삶의만족도_소속집단, Treatment('그렇지않다'))[T.그렇다]",0.896738,0.716938,1.121631


###### 개인, 소속집단 빼고

In [17]:
model_3_per_pol = sm.Logit.from_formula('''LongWork2 ~ 나이 + C(성별,Treatment('여자')) + C(건강상태,Treatment('건강하지않다')) 
    + C(결혼상태, Treatment('미혼'))+ C(대학지역,Treatment('비수도권')) + C(전공일치,Treatment('맞지않다')) 
    + 학점 + C(재학중일자리경험,Treatment('없다')) + C(삶의만족도_관계,Treatment('그렇지않다')) 
    + C(업무만족도,Treatment('그렇지않다')) ''', df_1).fit()
model_3_per_pol.summary()

Optimization terminated successfully.
         Current function value: 0.519824
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              LongWork2   No. Observations:                 5568
Model:                          Logit   Df Residuals:                     5553
Method:                           MLE   Df Model:                           14
Date:                Tue, 11 Apr 2023   Pseudo R-squ.:                  0.1100
Time:                        23:39:09   Log-Likelihood:                -2894.4
converged:                       True   LL-Null:                       -3252.2
Covariance Type:            nonrobust   LLR p-value:                1.195e-143
===========================================================================================================
                                              coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                                 -10.2707      0.619    -16.587      0.000     -11.484      -9.057
C(성별, Treatment('여자'))[T.남자]                0.2361      0.077      3.079      0.002       0.086       0.386
C(건강상태, Treatment('건강하지않다'))[T.건강하다]        0.1446      0.113      1.281      0.200      -0.077       0.366
C(건강상태, Treatment('건강하지않다'))[T.보통이다]        0.0530      0.121      0.439      0.661      -0.184       0.290
C(결혼상태, Treatment('미혼'))[T.기혼]              0.2898      0.212      1.366      0.172      -0.126       0.706
C(대학지역, Treatment('비수도권'))[T.수도권]           0.2312      0.066      3.518      0.000       0.102       0.360
C(전공일치, Treatment('맞지않다'))[T.보통이다]          0.5383      0.085      6.325      0.000       0.372       0.705
C(전공일치, Treatment('맞지않다'))[T.잘맞다]           0.8963      0.078     11.462      0.000       0.743       1.050
C(재학중일자리경험, Treatment('없다'))[T.있다]         -0.1419      0.067     -2.124      0.034      -0.273      -0.011
C(삶의만족도_관계, Treatment('그렇지않다'))[T.그렇다]      0.2963      0.128      2.309      0.021       0.045       0.548
C(삶의만족도_관계, Treatment('그렇지않다'))[T.보통이다]     0.1253      0.144      0.871      0.384      -0.157       0.407
C(업무만족도, Treatment('그렇지않다'))[T.그렇다]         0.8070      0.103      7.840      0.000       0.605       1.009
C(업무만족도, Treatment('그렇지않다'))[T.보통이다]        0.3437      0.107      3.223      0.001       0.135       0.553
나이                                          0.3054      0.022     13.705      0.000       0.262       0.349
학점                                         -0.0647      0.071     -0.913      0.361      -0.203       0.074
===========================================================================================================
"""

In [18]:
print('==================================Odds Ratio=========================================')
# 오즈비 계산(Confidence interval 포함)
odds_ratios_3_per_pol = pd.DataFrame(
    {
        "OR": model_3_per_pol.params,
        "Lower CI": model_3_per_pol.conf_int()[0],
        "Upper CI": model_3_per_pol.conf_int()[1],
    }
)
odds_ratios_3_per_pol = np.exp(odds_ratios_3_per_pol)
odds_ratios_3_per_pol

==================================Odds Ratio=========================================


,OR,Lower CI,Upper CI
Intercept,0.000035,0.000010,0.000117
"C(성별, Treatment('여자'))[T.남자]",1.266356,1.089617,1.471762
"C(건강상태, Treatment('건강하지않다'))[T.건강하다]",1.155624,0.926262,1.441780
"C(건강상태, Treatment('건강하지않다'))[T.보통이다]",1.054479,0.832099,1.336290
"C(결혼상태, Treatment('미혼'))[T.기혼]",1.336143,0.881583,2.025082
"C(대학지역, Treatment('비수도권'))[T.수도권]",1.260076,1.107791,1.433294
"C(전공일치, Treatment('맞지않다'))[T.보통이다]",1.713137,1.449917,2.024141
"C(전공일치, Treatment('맞지않다'))[T.잘맞다]",2.450577,2.102360,2.856470
"C(재학중일자리경험, Treatment('없다'))[T.있다]",0.867751,0.761276,0.989118
"C(삶의만족도_관계, Treatment('그렇지않다'))[T.그렇다]",1.344929,1.045792,1.729630
